In [1]:
import sys

sys.path.append("../")

import pandas as pd
from dotenv import load_dotenv

load_dotenv()
import os
import sqlite3

DB_PATH = os.getenv("DB_PATH")
DATA_PATH = "data"

conn = sqlite3.connect(DB_PATH)

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

/Users/charlesdedampierre/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/charlesdedampierre/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
# Individuals Regions

df = pd.read_csv("../immaterial_index/results/df_individuals_score.csv", index_col=[0])
print(len(set(df.individual_wikidata_id)))

df[df['individual_name']=='Hesiod']

172951


,individual_wikidata_id,productive_year,decade,individual_name,score,region_name
385999,Q44233,-765.0,-760,Hesiod,147,Balkans
386000,Q44233,-765.0,-760,Hesiod,147,Eastern Europe
386001,Q44233,-765.0,-760,Hesiod,147,Greek World
386002,Q44233,-765.0,-760,Hesiod,147,mediterranean World


In [3]:
# Load works of individuals

df_ind_works = pd.read_sql_query("SELECT * FROM individual_created_work", conn)

df_count_work = (
    df_ind_works.groupby("individual_wikidata_id")["work_wikidata_id"]
    .count()
    .rename("count_works")
    .reset_index()
)
df_final = pd.merge(df, df_count_work, on="individual_wikidata_id", how="left")
df_final = df_final.fillna(0)  # When there is no works we add 0
df_final = df_final.drop('productive_year', axis=1)
df_final.to_csv(DATA_PATH + "/df_indi_works.csv")

## Add gender

In [4]:
df_gender = pd.read_sql_query("SELECT * FROM individual_gender", conn)
df_final_gender = pd.merge(df_gender[['individual_wikidata_id', 'gender']], df_final, on = 'individual_wikidata_id')
df_final_gender.to_csv(DATA_PATH + "/df_indi_works_gender.csv")

## Associate a category to CPs based on the type of their work

In [5]:
# df_work_type = pd.read_sql_query("SELECT * FROM created_work", conn)
# df_work_category = df_work_type[['work_wikidata_id', 'instance_label']].drop_duplicates()
# df_work_covariate = pd.merge(df_ind_works, df_work_category, on = ['work_wikidata_id'])
# df_work_covariate['instance_label'].value_counts().head(20)
# df_work_covariate.category.value_counts()

# # took the top 20 categories that occur the most and group them

# # Define mapping of original categories to new categories
# category_mapping = {
#     'painting': 'painting',
#     'biographical article': 'literary work',
#     'poem': 'literary work',
#     'print': 'print',
#     'literary work': 'literary work',
#     'version, edition, or translation': 'literary work',
#     'drawing': 'drawing',
#     'encyclopedia article': 'literary work',
#     'article': 'literary work',
#     'photograph': 'photograph',
#     'publication': 'literary work',
#     'watercolor painting': 'painting',
#     'scholarly article': 'literary work',
#     'sculpture': 'sculpture',
#     'engraving': 'print',
#     'poetry': 'literary work',
#     'written work': 'literary work',
#     'etching print': 'print',
# }
# df_work_covariate['category']  = df_work_covariate['instance_label'].apply(lambda x:category_mapping.get(x, None)) # drop None otherwise
# df_work_covariate = df_work_covariate.dropna()
# df_work_covariate = df_work_covariate[['individual_wikidata_id', 'work_wikidata_id', 'category']].drop_duplicates()




# count_works = df_work_covariate.groupby(['individual_wikidata_id', 'category']).size().reset_index()

# # Handle indivuals who have two of the same category
# diverse_categories = count_works[count_works['individual_wikidata_id'].duplicated(keep=False)]
# filtered_df = diverse_categories.groupby('individual_wikidata_id').filter(lambda x: x[0].nunique() == 1)
# filtered_df = filtered_df.sample(frac=1, random_state=42).reset_index(drop=True)
# filtered_df = filtered_df.drop_duplicates('individual_wikidata_id', keep = 'first')
# filtered_df = filtered_df[['individual_wikidata_id', 'category']]

# # now do the rest
# # remove already handled individuals
# other_categories = count_works[~count_works['individual_wikidata_id'].isin(filtered_df['individual_wikidata_id'])]
# other_categories = other_categories.sort_values(['individual_wikidata_id', 0], ascending=(False, False))
# other_categories = other_categories.groupby('individual_wikidata_id').head(1)
# other_categories = other_categories[['individual_wikidata_id', 'category']]

# final_categories = pd.concat([other_categories, filtered_df])
# final_categories = final_categories.reset_index(drop=True)

# df_final_category = pd.merge(df_final, final_categories, on = ['individual_wikidata_id'])
# df_final_category.to_csv(DATA_PATH + "/df_indi_works_category.csv")

# len(set(final_categories.individual_wikidata_id)) == 40732

# final_categories.category.value_counts()

## Extract Occupations

In [6]:
df_occupations = pd.read_sql_query("SELECT * FROM individual_occupations", conn)
df_occupations.groupby('individual_wikidata_id')['occupations_wikidata_id'].count().mean()

top_occupations= list(df_occupations.occupations_name.value_counts().head(50).index)

In [7]:
occupation_categories = {
    'painter': 'Painter',
    'poet': 'Writer',
    'composer': 'Musician',
    #'university teacher': 'Educator',
    'historian': 'Writer',
    'theologian': 'Writer',
    'architect': 'Architect',
    'sculptor': 'Sculptor',
    'translator': 'Writer',
    'botanist': 'Writer',
    #'artist': 'Visual Artist',
    'philosopher': 'Writer',
    'drawer': 'Painter',
    #'photographer': 'Visual Artist',
    'mathematician': 'Writer',
    #'pedagogue': 'Educator',
    'engraver': 'Sculptor',
    'playwright': 'Performing Artist',
    #'explorer': 'Adventurer',
    'singer': 'Musician',
    'musician': 'Musician',
    'chemist': 'Writer',
    'linguist': 'Writer',
    'organist': 'Musician',
    'astronomer': 'Writer',
    'entomologist': 'Writer',
    'editor': 'Writer',
    'philologist': 'Writer',
    'stage actor': 'Performing Artist',
    'archaeologist': 'Writer',
    'naturalist': 'Writer',
    'printmaker': 'Painter',
    'physicist': 'Writer',
    'opera singer': 'Musician',
    'novelist': 'Writer',
    'conductor': 'Musician',
    'classical philologist': 'Writer',
    #'professor': 'Educator',
    'cartographer': 'Writer',
    'pianist': 'Musician',
    'economist': 'Writer',
    'copperplate engraver': 'Sculptor',
    'illustrator': 'Painter',
    'geologist': 'Writer',
    'zoologist': 'Writer',
    'scientific illustrator': 'Painter',
    'scientist': 'Writer',
    'geographer': 'Writer',
    'art historian': 'Painter',
    'violinist': 'Musician'
}

df_occupations['occupation'] = df_occupations['occupations_name'].apply(lambda x: occupation_categories.get(x, None))
df_occupations = df_occupations.dropna()
df_occupations.occupation.value_counts()

occupation
Writer               103061
Painter               53245
Musician              29267
Sculptor              13925
Architect             11599
Performing Artist      5736
Name: count, dtype: int64

In [8]:


df_unique_occupations = df_occupations[['individual_wikidata_id', 'occupation']].drop_duplicates()
df_unique_occupations = df_unique_occupations.sample(frac=1, random_state=42).reset_index(drop=True)
df_unique_occupations = df_unique_occupations.drop_duplicates('individual_wikidata_id', keep = 'first')

df_final_occupation = pd.merge(df_final, df_unique_occupations, on = 'individual_wikidata_id')
df_final_occupation = df_final_occupation[['individual_wikidata_id', 'decade', 'region_name', 'count_works', 'occupation']].drop_duplicates()


# Change paintings
#df_final_occupation['occupation'][(df_final_occupation['occupation'] == 'Painter') & (df_final_occupation['decade'] >= 1600)] = 'Painting Post-1600'
#df_final_occupation['occupation'][(df_final_occupation['occupation'] == 'Painter') & (df_final_occupation['decade'] < 1600)] = 'Painting Pre-1600'

In [9]:
df_final_occupation.occupation.value_counts()

occupation
Writer               213332
Painter              129267
Musician              55327
Sculptor              32894
Architect             29531
Performing Artist     13618
Name: count, dtype: int64

In [10]:

df_final_occupation.to_csv(DATA_PATH + "/df_indi_works_occupations.csv")

In [11]:
df_final_occupation[df_final_occupation['count_works']<=2].occupation.value_counts()

occupation
Writer               192852
Painter               90296
Musician              54687
Architect             28325
Sculptor              26203
Performing Artist     12211
Name: count, dtype: int64

In [12]:
# df_clean_gdp = pd.read_sql_query("SELECT * FROM gdp_clean", conn)

# region_code = pd.read_csv('../environnement_data/region_code_region_name.csv')
# df_clean_gdp = pd.merge(df_clean_gdp, region_code, on  = 'region_code')


# regions_clean = list(set(df_clean_gdp["region_name"]))

# df = df[df["region_name"].isin(regions_clean)]
# df = df[df["decade"] >= min(df_clean_gdp.year)]


# df_final = pd.merge(df, df_count_work, on="individual_wikidata_id", how="left")
# df_final = df_final.fillna(0)  # When there is no works we add 0
# df_final.to_csv(DATA_PATH + "/df_indi_works_clean_gdp.csv")